# Spin Qubit Pulse Sequences with the HDAWG

This notebook shows you how to use pulse sequences with an HDAWG to realize various experiments. While the sequences are mostly general, they are adapted to those typically used for spin qubits by adding the gate pulsing to control between Coulomb and spin blockade.

List of experiemnts
* Rabi - sweep length of burst
* Ramsey variant 1 - sweep delay with constant burst duration
* Ramsey variant 2 - sweep burst duration at constant delay
* Ramsey variant 3 - sweep phase of second burst and delay between bursts

## 0. General Imports

In [ ]:
%config IPCompleter.greedy=True

import numpy as np

from laboneq.simple import *

DO_EMULATION = True  # run in emulation mode by default

## 1. Device Setup
### 1.1 Calibration

In [ ]:
def calibrate_devices(device_setup):
    device_setup.logical_signal_groups["q0"].logical_signals[
        "drive_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q0_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.0, 0.0],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )

### 1.2 Create device setup

In [ ]:
descriptor = """\
instruments:
  HDAWG:
  - address: DEV8431
    uid: device_hdawg
connections:
  device_hdawg:
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
    - rf_signal: q0/coulomb_line_1
      ports: [SIGOUTS/0]
    - rf_signal: q0/coulomb_line_2
      ports: [SIGOUTS/1]
"""

device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="your_ip_address",
    server_port=8004,
    setup_name="MySetup",
)
calibrate_devices(device_setup)

## 2. Rabi Experiment: sweep burst length

### 2.1 Define parameters for experiment

In [ ]:
## constant definition
LEN_COULOMB_CYCLE = 200e-9
LEN_READOUT = 2 * LEN_COULOMB_CYCLE
X90_DURATION = 10e-9

START = 0
STOP = LEN_COULOMB_CYCLE / 2
STEPS = 5
NUM_REP = 2

In [ ]:
## define length sweep parameter
length_sweep_parameter = LinearSweepParameter(
    uid="length_sweep", start=START, stop=STOP, count=STEPS
)

### 2.2 Experiment

In [ ]:
# define pulse shapes
coulomb_pulse = pulse_library.const(
    uid="coulomb_pulse", length=LEN_COULOMB_CYCLE / 2, amplitude=0.5
)
coulomb_readout = pulse_library.const(
    uid="coulomb_readout", length=LEN_READOUT, amplitude=1
)
drive_pulse = pulse_library.const(
    uid="rabi_drive_pulse", length=X90_DURATION, amplitude=1
)

In [ ]:
## Create Experiment
exp = Experiment(
    "Rabi with Coulomb pulse",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid=("shots"), count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(parameter=length_sweep_parameter):
        with exp.section(
            uid=("empty"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
        with exp.section(
            uid=("manipulation"),
            length=LEN_COULOMB_CYCLE / 2,
            alignment=SectionAlignment.RIGHT,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)
            exp.play(signal="drive", pulse=drive_pulse, length=length_sweep_parameter)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
            exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
        with exp.section(
            uid="outer_trigger",
            length=LEN_READOUT,
            trigger={"drive": {"state": 1}},
            alignment=SectionAlignment.RIGHT,
        ):
            with exp.section(
                uid="inner_trigger",
                length=LEN_READOUT - 100e-9,
                trigger={"drive": {"state": 2}},
            ):
                exp.reserve(signal="drive")

### 2.3 Signal mapping

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 2.4 Session

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

In [ ]:
if not session.connection_state.emulated:
    instrument_serial = device_setup.instrument_by_uid("device_hdawg").address
    device = session.devices[instrument_serial]
    device.triggers.out[2].delay(23.9e-9)

### 2.5 Run

In [ ]:
session.run(exp)

### 2.6 View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Spin qubit length Rabi", compiled_exp)

## 3. Ramsey Experiment I: sweep wait time at constant burst length
### 3.1 Define parameters for experiment

In [ ]:
START = 0
STOP = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS = 3
NUM_REP = 20

In [ ]:
## Define sweep parameter
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START, stop=STOP, count=STEPS
)

### 3.2 Experiment

In [ ]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="sweep", parameter=sweep_delay):
        with exp.section(
            uid="qubit_excitation",
            alignment=SectionAlignment.RIGHT,
            length=LEN_COULOMB_CYCLE,
        ):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse)
            exp.delay(signal="drive", time=sweep_delay)
            exp.play(signal="drive", pulse=drive_pulse)
        with exp.section(uid="qubit_readout", length=LEN_READOUT):
            with exp.section(uid="readout_pulses"):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 3.3 Signal mapping

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=500e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 3.4 Connect Session and Run

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

### 3.5 View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant I", compiled_exp)

## 4. Ramsey Experiment II: sweep burst length at constant wait time
### 4.1 Define parameters for experiment

In [ ]:
# define constant delay
T_DELAY = 40e-9

In [ ]:
## parameters for experiment
START = 0
STOP = (LEN_COULOMB_CYCLE / 2 - T_DELAY) / 2
STEPS = 5
NUM_REP = 2

### 4.2 Experiment

In [ ]:
## Define sweep parameter
sweep_length = LinearSweepParameter(
    uid="pulse_length_sweep", start=START, stop=STOP, count=STEPS
)

In [ ]:
## Create Experiment
exp = Experiment(
    "Ramsey variant II",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.SEQUENTIAL
):
    with exp.sweep(uid="length_sweep", parameter=sweep_length):
        with exp.section(uid="qubit_excitation", alignment=SectionAlignment.RIGHT):
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
            exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
            exp.delay(signal="drive", time=T_DELAY)
            exp.play(signal="drive", pulse=drive_pulse, length=sweep_length)
        with exp.section(
            uid="qubit_readout", alignment=SectionAlignment.RIGHT, length=LEN_READOUT
        ):
            with exp.section(uid="readout_pulses"):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 4.3 Signal Mapping

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=100e6,
        modulation_type=ModulationType.HARDWARE,
    )
)

### 4.4 Connect Session and Run

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

### 4.5 View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant II", compiled_exp)

## 5. Ramsey Experiment III: 2D sweep, wait time vs phase of second pulse
### 5.1 Define parameters for experiment

In [ ]:
## constant definition
X90_DURATION = 10e-9  # [s]

START = 0
STOP = 2 * np.pi
STEPS = 5
NUM_REP = 2

START_DELAY = 0
STOP_DELAY = LEN_COULOMB_CYCLE / 2 - 2 * X90_DURATION
STEPS_DELAY = 3

In [ ]:
## define phase sweep parameter
sweep_phase = LinearSweepParameter(
    uid="phase_sweep", start=START, stop=STOP, count=STEPS
)
sweep_delay = LinearSweepParameter(
    uid="Ramsey_delay", start=START_DELAY, stop=STOP_DELAY, count=STEPS_DELAY
)

print(sweep_phase.values / np.pi)

### 5.2 Experiment

In [ ]:
## Create Experiment
exp = Experiment(
    "Ramsey variant I",
    signals=[
        ExperimentSignal("drive"),
        ExperimentSignal("coulomb_1"),
        ExperimentSignal("coulomb_2"),
    ],
)

# define experiment
with exp.acquire_loop_rt(
    uid="shots", count=NUM_REP, averaging_mode=AveragingMode.CYCLIC
):
    with exp.sweep(uid="sweep_delay", parameter=sweep_delay):
        with exp.sweep(uid="sweep_phase", parameter=sweep_phase):
            with exp.section(
                uid="qubit_excitation",
                alignment=SectionAlignment.RIGHT,
                length=LEN_COULOMB_CYCLE,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_1", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.5)
                exp.play(signal="coulomb_2", pulse=coulomb_pulse, amplitude=0.75)

                exp.play(signal="drive", pulse=drive_pulse, set_oscillator_phase=0)
                exp.delay(signal="drive", time=sweep_delay)
                exp.play(
                    signal="drive",
                    pulse=drive_pulse,
                    increment_oscillator_phase=sweep_phase,
                )
            with exp.section(
                uid="qubit_readout",
                alignment=SectionAlignment.RIGHT,
                length=LEN_READOUT,
            ):
                exp.play(signal="coulomb_1", pulse=coulomb_readout, amplitude=0.3)
                exp.play(signal="coulomb_2", pulse=coulomb_readout, amplitude=0.3)
            with exp.section(
                uid="outer_trigger",
                length=LEN_READOUT,
                trigger={"drive": {"state": 1}},
                alignment=SectionAlignment.RIGHT,
            ):
                with exp.section(
                    uid="inner_trigger",
                    length=LEN_READOUT - 100e-9,
                    trigger={"drive": {"state": 2}},
                ):
                    exp.reserve(signal="drive")

### 5.3 Signal Mapping

In [ ]:
# define signal maps for different qubits
map_q0 = {
    "drive": "/logical_signal_groups/q0/drive_line",
    "coulomb_1": "/logical_signal_groups/q0/coulomb_line_1",
    "coulomb_2": "/logical_signal_groups/q0/coulomb_line_2",
}

# calibration  for qubit 0
calib_q0 = Calibration()
calib_q0["drive"] = SignalCalibration(
    oscillator=Oscillator(
        frequency=0,
        modulation_type=ModulationType.SOFTWARE,
    )
)

print("Set modulation frequency to 0 Hz to better observe the phase sweep.")

### 5.4 Connect Session and Run

In [ ]:
# create and connect to session
session = Session(device_setup=device_setup)
session.connect(do_emulation=DO_EMULATION)

# set experiment calibration and signal map
exp.set_calibration(calib_q0)
exp.set_signal_map(map_q0)

session.run(exp)

### 5.5 View experiment in pulse sheet viewer

In [ ]:
# use pulse sheet viewer to display the pulse sequence - only recommended for small number of averages and sweep steps to avoid performance issues
compiled_exp = session.compiled_experiment
show_pulse_sheet("Ramsey variant III", compiled_exp)